In [1]:
from bs4 import BeautifulSoup
import requests
import json
import time
import pandas as pd
import os

In [2]:
match_names = ['Final', 'Third-place', 'Semi-finals-1', 'Semi-finals-2', 'Quarter-final-1', 'Quarter-final-2', 
               'Quarter-final-3', 'Quarter-final-4']

In [3]:
def get_names(match, url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find_all('div', class_ = 'table_wrapper')
    keys = [i.select('caption')[0].text for i in table[:4]]

    for i in range(4):
        match[keys[i]] = pd.read_html(str(table[i]))[0]

In [4]:
def get_match(event, event_stats, url):
    
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    matches = soup.find_all('div', class_ = 'match-summary')

    for i in range(len(match_names)):

        match = match_names[i]
        event[match]={}
        event_stats[match]={}

        event[match]['winner'] = matches[i].select('a')[0].text
        event[match]['loser'] = matches[i].select('a')[2].text
        event[match]['winner_goal'] = matches[i].select('a')[1].text.split('–')[0]
        event[match]['loser_goal'] = matches[i].select('a')[1].text.split('–')[0]
        url_match = 'https://fbref.com' + matches[i].select('a')[1].get('href')

        get_names(event_stats[match], url_match)
        time.sleep(4)

In [5]:
WC_2007 = {}
WC_2007_stats = {}

In [6]:
get_match(WC_2007, WC_2007_stats, 'https://fbref.com/en/comps/106/2007/2007-Womens-World-Cup-Stats')

In [8]:
for key in match_names:
    os.mkdir('WC_2007/'+key)

In [9]:
for key in WC_2007.keys():
    df = pd.DataFrame.from_dict(WC_2007[key],orient='index')
    df.to_csv('WC_2007/'+key+'/result.csv', index = False)

In [10]:
for key in WC_2007.keys():
    for tag in WC_2007_stats[key].keys():
        df = WC_2007_stats[key][tag]
        df.to_csv('WC_2007/'+key+'/'+tag+'.csv', index = False)